In [218]:
from polybinder.system import System, Initializer
from polybinder.simulate import Simulation
import mbuild as mb
import hoomd
from mbuild.formats.hoomd_forcefield import to_hoomdsnapshot
import numpy as np

In [219]:
import warnings
warnings.filterwarnings("ignore")

In [220]:
## set parameters
dt = 0.003
tau_kT = 10
kT = 1.5

In [221]:
system_params = System(density=0.01,
                    molecule="PPS",
                    polymer_lengths=1,
                    n_compounds=5,
                    para_weight=1.0,
                   )

In [222]:
PPS_system_saved = Initializer(system=system_params,
                        system_type='stack',
                        remove_hydrogens=True,
                        n_rigid_beads=system_params.polymer_lengths[0] * system_params.n_compounds[0],
                        rigid_bead_len=7,
                        rigid_bead_names=['R1', 'R2'],
                        separation=0.6,
                        forcefield="opls",
                        save_parmed=False
                              )

Removing hydrogens and adjusting heavy atoms
---------------------------------------------------------------
make rigid snap...


In [223]:
import os
from datetime import datetime
log_dir = "/home/marjanalbooyeh/logs/pps_rigid/"
now = datetime.now()
log_path = os.path.join(log_dir, now.strftime("%Y-%m-%d-%H:%M:%S")) + '/'


if not os.path.exists(log_path):
    os.mkdir(log_path)

In [224]:
simulation = Simulation(system=PPS_system_saved, dt=dt, tau_kt=tau_kT, logs_path=log_path, gsd_write=1000, log_write=1000)

Processing LJ and QQ
No charged groups found, ignoring electrostatics
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing RB torsions
body_name:  R2
body_name:  R2
body_name:  R2
body_name:  R2
body_name:  R2


In [123]:
simulation.sim.operations.integrator.forces[0].nlist.exclusions.extend(['body'])

In [225]:
for i, force in enumerate(simulation.sim.operations.integrator.forces):
#     if isinstance(force, hoomd.md.pair.LJ):
#         for t in snapshot.particles.types:
#             for r in ['R1', 'R2']:
#                 force.params[(r, t)] = dict(epsilon=0, sigma=0)
#                 force.r_cut[(r, t)] = 0
    if isinstance(force, hoomd.md.special_pair.LJ):
        for k in force.params.keys():
            force.params[k] = dict(epsilon=0., sigma=0.)
    if isinstance(force, hoomd.md.bond.Harmonic):
        for k in force.params.keys():
            force.params[k] = dict(k=0, r0=0)
    if isinstance(force, hoomd.md.angle.Harmonic):
        for k in force.params.keys():
            force.params[k] = dict(t0=0, k=0)
    if isinstance(force, hoomd.md.dihedral.Harmonic):
        for k in force.params.keys():
            force.params[k] = dict(k=0, d=0, n=0, phi0=0)

In [12]:
simulation.quench(n_steps=1, kT=kT)

In [226]:
simulation.quench(n_steps=10e6, kT=kT)

*Warning*: angle.harmonic: specified K <= 0
*Warning*: angle.harmonic: specified t_0 <= 0
*Warning*: angle.harmonic: specified K <= 0
*Warning*: angle.harmonic: specified t_0 <= 0


In [227]:
log_path

'/home/marjanalbooyeh/logs/pps_rigid/2023-03-09-18:47:47/'

In [72]:
simulation.sim.state.get_snapshot().particles.typeid

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3,
       2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2,
       2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2,
       2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2], dtype=uint32)

In [228]:

simulation.sim.operations.writers[-1].file.close()

### prepare data

In [229]:
import h5py
import numpy as np
import pandas as pd


In [230]:
rigid_traj_path = log_path + 'rigid_traj.h5'
f = h5py.File(rigid_traj_path, 'r')

In [231]:
traj_df = pd.DataFrame()
for frame in f.keys():
    dset = dict(f[frame])
    dset = {key: np.array(value) for key, value in dset.items()}
    dset['timestep'] = f[frame].attrs['timestep']
    traj_df = traj_df.append(dset, ignore_index=True)

In [232]:
f.close()

In [233]:
positions = np.asarray(traj_df["position"].tolist())
orientations = np.asarray(traj_df["orientation"].tolist())

### calculate rel. position to all other particles

In [234]:
import rowan

In [235]:
num_particles = positions.shape[1]
rel_positions = np.zeros((positions.shape[0], positions.shape[1], positions.shape[1] - 1, positions.shape[2] + 1))
rel_orientations = np.zeros((orientations.shape[0],orientations.shape[1], orientations.shape[1] - 1, orientations.shape[2] + 1))
for j, (frame_positions, frame_orientations) in enumerate(zip(positions, orientations)):
    for i in range(num_particles):
        other_particles_idx = list(range(num_particles))
        other_particles_idx.remove(i)
        rel_positions[j,i, :, :3] = frame_positions[i] - frame_positions[other_particles_idx]
        rel_positions[j, i, :, 3] = np.linalg.norm(frame_positions[i] - frame_positions[other_particles_idx], axis=1)

            
        rel_orientations[j, i, :, :4] = rowan.geometry.riemann_log_map(frame_orientations[i], frame_orientations[other_particles_idx])
        rel_orientations[j, i, :, 4] = rowan.geometry.sym_intrinsic_distance(frame_orientations[i], frame_orientations[other_particles_idx])

In [236]:
rel_orientations.shape

(10001, 5, 4, 5)

In [77]:
# rel_positions = np.zeros_like(positions)
# rel_positions[:, 0, :] = positions[:, 0, :] - positions[:, 1, :]
# rel_positions[:, 1, :] = positions[:, 1, :] - positions[:, 0, :]

In [78]:
# rel_orientations = np.zeros_like(orientations)
# rel_orientations[:, 0, :] = orientations[:, 0, :] - orientations[:, 1, :]
# rel_orientations[:, 1, :] = orientations[:, 1, :] - orientations[:, 0, :]

In [237]:
columns = [
    "position",
    "orientation",
    "net_force",
    "net_torque",
]

import pandas as pd
new_traj_df = pd.DataFrame(columns=columns)

In [238]:
new_traj_df["position"] = rel_positions.tolist()
new_traj_df["orientation"] = rel_orientations.tolist()
new_traj_df["net_force"] = traj_df["net_force"].tolist()
new_traj_df["net_torque"] = traj_df["net_torque"].tolist()

In [239]:
#shuffle order of frames
new_traj_df = new_traj_df.sample(frac=1).reset_index(drop=True)


In [240]:
test_frac = 0.1
val_frac = 0.1

dataset_len = new_traj_df.shape[0]
test_len = int(dataset_len * test_frac)
val_len = int(dataset_len * val_frac)


In [241]:
test_df = new_traj_df.loc[:test_len]

val_df = new_traj_df.loc[test_len: test_len + val_len]

train_df = new_traj_df.loc[test_len + val_len:]

In [242]:
train_df.shape

(8001, 4)

In [243]:
target_data_path = os.path.join(log_path, 'dataset')

if not os.path.exists(target_data_path):
    os.mkdir(target_data_path)

In [244]:
train_df.to_pickle(os.path.join(target_data_path, 'train.pkl'))
val_df.to_pickle(os.path.join(target_data_path, 'val.pkl'))
test_df.to_pickle(os.path.join(target_data_path,'test.pkl'))

In [245]:
target_data_path

'/home/marjanalbooyeh/logs/pps_rigid/2023-03-09-18:47:47/dataset'